In [1]:
# takes 2m30s for 25k rows (10m exploded rows) 100GB RAM. 7m for 1m rows (48m exploded rows) 300GB RAM.
# perform experiments on bidding tables.

# if git commit fails due to large size, rollback one git commit using: git reset --soft HEAD~1

# previous steps:
# acbl_club_results_download_to_json.ipynb
# acbl_club_results_json_to_sql.ipynb creates sqlite:///acbl_club_results.sqlite 
# acbl_club_results_sql_to_hand_records.ipynb creates acbl_club_results_hand_records.pkl
# acbl_club_results_hand_records_clean.ipynb creates acbl_club_results_hand_records_cleaned.pkl
# acbl_club_results_hand_records_augment.ipynb creates acbl_club_hand_records_augmented.pkl
# bbo/bbo-downloader/BBO-Downloader.py downloads BBO's .lin files. lin files contain a user's result of a board (hand, bidding and card play).
# bbo/BBO-Bidding/bbo_parse_lin_files.ipynb creates bbo_bidding_sequences_table.py (don't forget to install requirements.txt).
# bbo/BBO-Bidding/create_bbo_bidding_dicts.ipynb bbo_bidding_dicts.pkl
# bbo/BBO-Bidding/bbo_generate_bids.ipynb creates bbo_bidding_issues_dicts.pkl and bbo_bidding_sequences_dfs.parquet and bbo_bidding_sequences_dicts.pkl
# BBO-Bidding/bbo_vet_bids.ipynb vets hands
# acbl_club_results_hand_records_bidding_BBO.ipynb creates acbl_club_bidding_hashed_df.parquet and acbl_club_bidding_unique_rows_df.pkl
# acbl_club_results_hand_records_bidding_BBO_augment.ipynb creates various bidding files.

# next steps:
# acbl_club_results_hand_records_bidding_BBO_experiments1.ipynb


In [2]:
import polars as pl
import pathlib
import pickle
import re
from collections import defaultdict
import sys
import time
import gc

program_start_time = time.time()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2024-09-25 15:57:38


In [3]:
sys.path.append(str(pathlib.Path.cwd().parent.joinpath('mlBridgeLib')))
sys.path
import mlBridgeLib

In [4]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [5]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')
savedModelsPath = acblPath.joinpath('SavedModels')
# create parent directories in case they don't already exist.
savedModelsPath.mkdir(parents=True, exist_ok=True)

In [6]:
bboPath = rootPath.joinpath('bbo')
dataPath = bboPath.joinpath('data')

In [7]:
# restrict processing to dealer 'N' to minimize file and memory usage.
dealer = 'N'

In [8]:
# take 20s. 1.8GB file.
bbo_parsed_lin_files_df_filename = f'bbo_parsed_lin_files_df.parquet'
bbo_parsed_lin_files_df_file = dataPath.joinpath(bbo_parsed_lin_files_df_filename)
df =pl.read_parquet(bbo_parsed_lin_files_df_file)
print(f"Loaded: {bbo_parsed_lin_files_df_filename}: shape:{df.shape} size:{bbo_parsed_lin_files_df_file.stat().st_size}")

Loaded: bbo_parsed_lin_files_df.parquet: shape:(14224835, 15) size:2309159310


In [9]:
df

lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN
str,i32,str,u8,str,str,str,str,str,str,list[str],list[str],list[str],list[u8],list[u8]
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",1,"""N""","""None""","""S68H27TAD7TJQCQKA""","""S79JQKAHD24AC235J""","""S5TH34689JD59KC67""","""S234H5QKD368C489T""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C2"", ""C6"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",2,"""E""","""NS""","""S6TQKH5D5JKAC237Q""","""S23JH67JKD27TQC6A""","""S5789H348QD49C45T""","""S4AH29TAD368C89JK""","[""1C"", ""1S"", … ""p""]","[""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", ""One-level overcall -- 5+ !S; 8-17 HCP; 9-19 total points"", … """"]","[""DA"", ""D7"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",4,"""W""","""Both""","""S4569KAHAD6JQC45A""","""S3H678QD2349C679T""","""S28QH245TJD8TKC3Q""","""S7TJH39KD57AC28JK""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""SA"", ""S3"", … ""DJ""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",5,"""N""","""NS""","""S9AHD3467TQKAC3KA""","""SJQH25JQKAD58JC56""","""S3468KH89TD9C247J""","""S257TH3467D2C89TQ""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C6"", ""C7"", … ""HQ""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",6,"""E""","""EW""","""S48JH36QKAD59KC9K""","""S279KH78TDJQAC26T""","""S36TAH24D46TC45JA""","""S5QH59JD2378C378Q""","[""p"", ""1C"", … ""p""]","["""", ""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", … """"]","[""DQ"", ""D4"", … ""SK""]","[1, 1, … 13]","[1, 2, … 4]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""e:\bridge\data\bbo\data\_miki_…",-1,"""_miki_""",4,"""W""","""Both""","""S6JH8JQKD8KC46JQA""","""SKAH39TD2479QC3TK""","""S235QH245D6TJAC29""","""S4789TH67AD35C578""","[""1D"", ""p"", … ""p""]","[""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", """", … """"]","[""SA"", ""S2"", … ""S8""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\_miki_…",-1,"""_miki_""",5,"""N""","""NS""","""S5TAHJQKD5AC368QA""","""S368QKH67TD26KC4J""","""S9H9D34789TJC57TK""","""S247JH23458ADQC29""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""H9"", ""HA"", … ""C6""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\_miki_…",-2,"""_miki_""",6,"""E""","""EW""","""S2JH45QKD9JKAC3KA""","""S347TQKH26ADQC46T""","""S8H3789JD24678TC9""","""S569AHTD35C2578JQ""","[""p"", ""1D"", … ""p""]","["""", ""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", … """"]","[""C4"", ""C9"", … ""H4""]","[1, 1, … 6]","[1, 2, … 4]"


In [10]:
df.null_count()

lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
df = df.drop_nulls()
df.null_count()

lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
df['Error'].value_counts().sort('Error') # range(-2 to 210)


Error,count
i32,u32
-2,3505094
-1,10661297
34,86
36,88
38,81
…,…
216,1
220,1
224,1


In [13]:
df['Username'].value_counts().sort('count',descending=True)

Username,count
str,u32
"""imacooljet""",55559
"""icewing""",40530
"""zhenya__S""",38034
"""jant88""",37619
"""Olgun bey""",35200
…,…
"""~~M35768""",1
"""~~M58083""",1
"""~~M46147""",1


In [14]:
df.filter(df['Username'].eq('bsalita'))

lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN
str,i32,str,u8,str,str,str,str,str,str,list[str],list[str],list[str],list[u8],list[u8]
"""e:\bridge\data\bbo\data\bsalit…",-2,"""bsalita""",1,"""N""","""None""","""S9QKH7TD568QAC49Q""","""S357AH258AD3TJC7J""","""S46H346QKD27KC5TK""","""S28TJH9JD49C2368A""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""CJ"", ""C5"", … ""D3""]","[1, 1, … 8]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\bsalit…",-2,"""bsalita""",2,"""E""","""NS""","""S4JQAH7D4567C35QA""","""S23TH6TD389QAC47T""","""S678H489JQKADTC29""","""S59KH235D2JKC68JK""","[""p"", ""1D"", … ""p""]","["""", ""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", … """"]","[""H2"", ""H7"", … ""SK""]","[1, 1, … 8]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\bsalit…",-2,"""bsalita""",3,"""S""","""EW""","""S2JQKAH6JD23JC89K""","""S345H79D468AC2467""","""S679TH3458KD59KCJ""","""S8H2TQAD7TQC35TQA""","[""1S"", ""p"", … ""p""]","[""Major suit opening -- 5+ !S; 11-21 HCP; 12-22 total points"", """", … """"]","[""H9"", ""H3"", … ""DQ""]","[1, 1, … 9]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\bsalit…",-2,"""bsalita""",4,"""W""","""Both""","""S2QH8DJAC23679JKA""","""S467JKAH9TD8TQC8T""","""S89H235JKAD2567C5""","""S35TH467QD349KC4Q""","[""2S"", ""p"", … ""p""]","[""Weak two bid -- 1-4 !C; 1-3 !D; 1-3 !H; 6+ !S; 6-10 HCP; 7+ total points"", """", … """"]","[""SK"", ""S8"", … ""CQ""]","[1, 1, … 4]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\bsalit…",-1,"""bsalita""",5,"""N""","""NS""","""S29QH27TD3AC29JQA""","""S48KH46KD2456TKCT""","""S57TJH89QAD8JQC67""","""S36AH35JD79C3458K""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C7"", ""C3"", … ""ST""]","[1, 1, … 13]","[1, 2, … 4]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""e:\bridge\data\bbo\data\bsalit…",-1,"""bsalita""",12,"""W""","""NS""","""S357TAH367QAD4C3K""","""S2689QHKD68TJQKCT""","""SJKH89JD57AC289JQ""","""S4H245TD239C4567A""","[""1D"", ""p"", … ""p""]","[""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", """", … """"]","[""CQ"", ""CA"", … ""HQ""]","[1, 1, … 13]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\bsalit…",-2,"""bsalita""",1,"""N""","""None""","""S569JQH2AD2AC56QA""","""S78H3789QD57C379K""","""S23TKAH6TJKD6C28J""","""S4H45D3489TJQKC4T""","[""1S"", ""4D"", … ""p""]","[""Major suit opening -- 5+ !S; 11-21 HCP; 12-22 total points"", ""Aggressive weak jump overcall -- 7+ !D; 11- HCP; 3+ total points"", … """"]","[""DK"", ""DA"", … ""D6""]","[1, 1, … 1]","[1, 2, … 4]"
"""e:\bridge\data\bbo\data\bsalit…",-2,"""bsalita""",2,"""E""","""NS""","""S8QAH78D2JKC479JA""","""S23KH69D578C2368K""","""S46H235JQKD46TACQ""","""S579TJH4TAD39QC5T""","[""p"", ""1N"", … ""p""]","["""", ""notrump opener. Could have 5M. -- 2-5 !C; 2-5 !D; 2-5 !H; 2-5 !S; 15-"", … """"]","[""D5"", ""D4"", … ""SJ""]","[1, 1, … 11]","[1, 2, … 4]"


In [15]:
df['Board'].value_counts().sort('count',descending=True)


Board,count
u8,u32
1,1480411
2,1462935
3,1451167
4,1439869
5,1421485
…,…
19,223
23,9
22,9


In [16]:
df['Dealer'].replace({3:'N',4:'E',1:'S',2:'W'}).value_counts()

Dealer,count
str,u32
"""W""",3494408
"""S""",3534894
"""N""",3620751
"""E""",3574782


In [17]:
df['Vul'].replace({'o':'None','n':'NS','e':'EW','b':'Both'}).value_counts()

Vul,count
str,u32
"""None""",3547776
"""EW""",3570509
"""Both""",3532339
"""NS""",3574211


In [18]:
df['Auction'].value_counts().sort('count',descending=True)


Auction,count
list[str],u32
"[""1N"", ""p"", … ""p""]",145336
"[""p"", ""1N"", … ""p""]",106262
"[""p"", ""p"", … ""p""]",104611
"[""p"", ""p"", … ""p""]",66631
"[""1N"", ""p"", … ""p""]",52186
…,…
"[""1C"", ""d"", … ""p""]",1
"[""1D"", ""1S"", … ""p""]",1
"[""p"", ""p"", … ""p""]",1


In [19]:
df['Announcement'].value_counts().sort('count',descending=True)


Announcement,count
list[str],u32
"[""notrump opener. Could have 5M. -- 2-5 !C; 2-5 !D; 2-5 !H; 2-5 !S; 15-"", """", … """"]",139602
"["""", ""notrump opener. Could have 5M. -- 2-5 !C; 2-5 !D; 2-5 !H; 2-5 !S; 15-"", … """"]",102916
"["""", """", … """"]",101215
"["""", """", … """"]",66154
"[""notrump opener. Could have 5M. -- 2-5 !C; 2-5 !D; 2-5 !H; 2-5 !S; 15-"", """", … """"]",50594
…,…
"["""", """", … """"]",1
"["""", ""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", … """"]",1
"[""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", ""Takeout double -- 3-5 !C; 2- !D; 3-4 !H; 3-4 !S; 12+ total points"", … """"]",1


In [20]:
#df['Claim'].value_counts().sort('count',descending=True)

In [21]:
# takes 3m
#pbns = [r[0]+':'+' '.join(['.'.join(list(map(lambda x: x[::-1], re.split('S|H|D|C', hh)))[1:]) for hh in r[1:]]) for r in df.select(pl.col('Dealer',r'^Hand_[NESW]$')).rows()]
pbns = ['N:'+' '.join(['.'.join(list(map(lambda x: x[::-1], re.split('S|H|D|C', hh)))[1:]) for hh in r]) for r in df.select(pl.col(r'^Hand_[NESW]$')).rows()]
pbns

['N:86.AT72.QJT7.AKQ AKQJ97..A42.J532 T5.J98643.K95.76 432.KQ5.863.T984',
 'N:KQT6.5.AKJ5.Q732 J32.KJ76.QT72.A6 9875.Q843.94.T54 A4.AT92.863.KJ98',
 'N:AK9654.A.QJ6.A54 3.Q876.9432.T976 Q82.JT542.KT8.Q3 JT7.K93.A75.KJ82',
 'N:A9..AKQT7643.AK3 QJ.AKQJ52.J85.65 K8643.T98.9.J742 T752.7643.2.QT98',
 'N:J84.AKQ63.K95.K9 K972.T87.AQJ.T62 AT63.42.T64.AJ54 Q5.J95.8732.Q873',
 'N:AJ743.Q7.A653.Q4 K982.K842.J2.KJ6 65.A.KQT84.97532 QT.JT9653.97.AT8',
 'N:AQT5.764.AQ2.A86 K982.T.T76.JT752 J6.KJ932.J5.KQ93 743.AQ85.K9843.4',
 'N:2.AT95.AKQ654.QT A76.KQJ7.T83.985 KQT9.42.J2.AKJ74 J8543.863.97.632',
 'N:AQJ65.JT.AKT64.9 KT83.K4.J9753.AT 97.A97652.Q8.J62 42.Q83.2.KQ87543',
 'N:T5.963.AKQ95.A97 97.Q75.J432.T652 KQ86.A842.876.K4 AJ432.KJT.T.QJ83',
 'N:KJT.AKQJ.J7.Q953 942.8754.42.KT42 873.32.983.AJ876 AQ65.T96.AKQT65.',
 'N:AJ2.K98.A.AKT864 KQ984.2.T87542.J T763.QT763.J63.5 5.AJ54.KQ9.Q9732',
 'N:AK2.Q43.A85.AK63 73.AJT2.Q63.T742 T965.K97.KJ2.985 QJ84.865.T974.QJ',
 'N:AJ82.AJ82.987.K5 T94.QT9.KQJ2.J97 

In [22]:
df = df.with_columns(pl.Series('PBN',pbns,pl.String))
df


lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN,PBN
str,i32,str,u8,str,str,str,str,str,str,list[str],list[str],list[str],list[u8],list[u8],str
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",1,"""N""","""None""","""S68H27TAD7TJQCQKA""","""S79JQKAHD24AC235J""","""S5TH34689JD59KC67""","""S234H5QKD368C489T""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C2"", ""C6"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]","""N:86.AT72.QJT7.AKQ AKQJ97..A42…"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",2,"""E""","""NS""","""S6TQKH5D5JKAC237Q""","""S23JH67JKD27TQC6A""","""S5789H348QD49C45T""","""S4AH29TAD368C89JK""","[""1C"", ""1S"", … ""p""]","[""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", ""One-level overcall -- 5+ !S; 8-17 HCP; 9-19 total points"", … """"]","[""DA"", ""D7"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]","""N:KQT6.5.AKJ5.Q732 J32.KJ76.QT…"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",4,"""W""","""Both""","""S4569KAHAD6JQC45A""","""S3H678QD2349C679T""","""S28QH245TJD8TKC3Q""","""S7TJH39KD57AC28JK""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""SA"", ""S3"", … ""DJ""]","[1, 1, … 13]","[1, 2, … 4]","""N:AK9654.A.QJ6.A54 3.Q876.9432…"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",5,"""N""","""NS""","""S9AHD3467TQKAC3KA""","""SJQH25JQKAD58JC56""","""S3468KH89TD9C247J""","""S257TH3467D2C89TQ""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C6"", ""C7"", … ""HQ""]","[1, 1, … 13]","[1, 2, … 4]","""N:A9..AKQT7643.AK3 QJ.AKQJ52.J…"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",6,"""E""","""EW""","""S48JH36QKAD59KC9K""","""S279KH78TDJQAC26T""","""S36TAH24D46TC45JA""","""S5QH59JD2378C378Q""","[""p"", ""1C"", … ""p""]","["""", ""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", … """"]","[""DQ"", ""D4"", … ""SK""]","[1, 1, … 13]","[1, 2, … 4]","""N:J84.AKQ63.K95.K9 K972.T87.AQ…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""e:\bridge\data\bbo\data\_miki_…",-1,"""_miki_""",4,"""W""","""Both""","""S6JH8JQKD8KC46JQA""","""SKAH39TD2479QC3TK""","""S235QH245D6TJAC29""","""S4789TH67AD35C578""","[""1D"", ""p"", … ""p""]","[""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", """", … """"]","[""SA"", ""S2"", … ""S8""]","[1, 1, … 13]","[1, 2, … 4]","""N:J6.KQJ8.K8.AQJ64 AK.T93.Q974…"
"""e:\bridge\data\bbo\data\_miki_…",-1,"""_miki_""",5,"""N""","""NS""","""S5TAHJQKD5AC368QA""","""S368QKH67TD26KC4J""","""S9H9D34789TJC57TK""","""S247JH23458ADQC29""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""H9"", ""HA"", … ""C6""]","[1, 1, … 13]","[1, 2, … 4]","""N:AT5.KQJ.A5.AQ863 KQ863.T76.K…"
"""e:\bridge\data\bbo\data\_miki_…",-2,"""_miki_""",6,"""E""","""EW""","""S2JH45QKD9JKAC3KA""","""S347TQKH26ADQC46T""","""S8H3789JD24678TC9""","""S569AHTD35C2578JQ""","[""p"", ""1D"", … ""p""]","["""", ""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", … """"]","[""C4"", ""C9"", … ""H4""]","[1, 1, … 6]","[1, 2, … 4]","""N:J2.KQ54.AKJ9.AK3 KQT743.A62.…"


In [23]:
# takes 10s for 1m rows and 182MB file size.
bbo_hand_dict_filename = 'bbo_hand_dict.pkl'
bbo_hand_dict_file = dataPath.joinpath(bbo_hand_dict_filename)
with open(bbo_hand_dict_file,'rb') as f:
    hrs_d = pickle.load(f)
print(f"Loaded {bbo_hand_dict_filename}: len:{len(hrs_d)} size:{bbo_hand_dict_file.stat().st_size}")

Loaded bbo_hand_dict.pkl: len:887776 size:182913127


In [26]:
# takes 4h to create all new hands. 8m-20m if most hands are in dict. takes 10m to add column to df.
hands = [hrs_d[pbn]['Hands'] if pbn in hrs_d and 'Hands' in hrs_d[pbn] else mlBridgeLib.pbn_to_hands(pbn) for pbn in df['PBN']]
df = df.with_columns(pl.Series('Hands',hands,pl.List(pl.List(pl.String))))
df

lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN,PBN,Hands
str,i32,str,u8,str,str,str,str,str,str,list[str],list[str],list[str],list[u8],list[u8],str,list[list[str]]
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",1,"""N""","""None""","""S68H27TAD7TJQCQKA""","""S79JQKAHD24AC235J""","""S5TH34689JD59KC67""","""S234H5QKD368C489T""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C2"", ""C6"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]","""N:86.AT72.QJT7.AKQ AKQJ97..A42…","[[""86"", ""AT72"", … ""AKQ""], [""AKQJ97"", """", … ""J532""], … [""432"", ""KQ5"", … ""T984""]]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",2,"""E""","""NS""","""S6TQKH5D5JKAC237Q""","""S23JH67JKD27TQC6A""","""S5789H348QD49C45T""","""S4AH29TAD368C89JK""","[""1C"", ""1S"", … ""p""]","[""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", ""One-level overcall -- 5+ !S; 8-17 HCP; 9-19 total points"", … """"]","[""DA"", ""D7"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]","""N:KQT6.5.AKJ5.Q732 J32.KJ76.QT…","[[""KQT6"", ""5"", … ""Q732""], [""J32"", ""KJ76"", … ""A6""], … [""A4"", ""AT92"", … ""KJ98""]]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",4,"""W""","""Both""","""S4569KAHAD6JQC45A""","""S3H678QD2349C679T""","""S28QH245TJD8TKC3Q""","""S7TJH39KD57AC28JK""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""SA"", ""S3"", … ""DJ""]","[1, 1, … 13]","[1, 2, … 4]","""N:AK9654.A.QJ6.A54 3.Q876.9432…","[[""AK9654"", ""A"", … ""A54""], [""3"", ""Q876"", … ""T976""], … [""JT7"", ""K93"", … ""KJ82""]]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",5,"""N""","""NS""","""S9AHD3467TQKAC3KA""","""SJQH25JQKAD58JC56""","""S3468KH89TD9C247J""","""S257TH3467D2C89TQ""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C6"", ""C7"", … ""HQ""]","[1, 1, … 13]","[1, 2, … 4]","""N:A9..AKQT7643.AK3 QJ.AKQJ52.J…","[[""A9"", """", … ""AK3""], [""QJ"", ""AKQJ52"", … ""65""], … [""T752"", ""7643"", … ""QT98""]]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",6,"""E""","""EW""","""S48JH36QKAD59KC9K""","""S279KH78TDJQAC26T""","""S36TAH24D46TC45JA""","""S5QH59JD2378C378Q""","[""p"", ""1C"", … ""p""]","["""", ""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", … """"]","[""DQ"", ""D4"", … ""SK""]","[1, 1, … 13]","[1, 2, … 4]","""N:J84.AKQ63.K95.K9 K972.T87.AQ…","[[""J84"", ""AKQ63"", … ""K9""], [""K972"", ""T87"", … ""T62""], … [""Q5"", ""J95"", … ""Q873""]]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""e:\bridge\data\bbo\data\_miki_…",-1,"""_miki_""",4,"""W""","""Both""","""S6JH8JQKD8KC46JQA""","""SKAH39TD2479QC3TK""","""S235QH245D6TJAC29""","""S4789TH67AD35C578""","[""1D"", ""p"", … ""p""]","[""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", """", … """"]","[""SA"", ""S2"", … ""S8""]","[1, 1, … 13]","[1, 2, … 4]","""N:J6.KQJ8.K8.AQJ64 AK.T93.Q974…","[[""J6"", ""KQJ8"", … ""AQJ64""], [""AK"", ""T93"", … ""KT3""], … [""T9874"", ""A76"", … ""875""]]"
"""e:\bridge\data\bbo\data\_miki_…",-1,"""_miki_""",5,"""N""","""NS""","""S5TAHJQKD5AC368QA""","""S368QKH67TD26KC4J""","""S9H9D34789TJC57TK""","""S247JH23458ADQC29""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""H9"", ""HA"", … ""C6""]","[1, 1, … 13]","[1, 2, … 4]","""N:AT5.KQJ.A5.AQ863 KQ863.T76.K…","[[""AT5"", ""KQJ"", … ""AQ863""], [""KQ863"", ""T76"", … ""J4""], … [""J742"", ""A85432"", … ""92""]]"
"""e:\bridge\data\bbo\data\_miki_…",-2,"""_miki_""",6,"""E""","""EW""","""S2JH45QKD9JKAC3KA""","""S347TQKH26ADQC46T""","""S8H3789JD24678TC9""","""S569AHTD35C2578JQ""","[""p"", ""1D"", … ""p""]","["""", ""Minor suit opening -- 3+ !D; 11-21 HCP; 12-22 total points"", … """"]","[""C4"", ""C9"", … ""H4""]","[1, 1, … 6]","[1, 2, … 4]","""N:J2.KQ54.AKJ9.AK3 KQT743.A62.…","[[""J2"", ""KQ54"", … ""AK3""], [""KQT743"", ""A62"", … ""T64""], … [""A965"", ""T"", … ""QJ8752""]]"


In [33]:
df.head(2)

lin_file,Error,Username,Board,Dealer,Vul,Hand_N,Hand_E,Hand_S,Hand_W,Auction,Announcement,Card,Trick,PCN,PBN,Hands
str,i32,str,u8,str,str,str,str,str,str,list[str],list[str],list[str],list[u8],list[u8],str,list[list[str]]
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",1,"""N""","""None""","""S68H27TAD7TJQCQKA""","""S79JQKAHD24AC235J""","""S5TH34689JD59KC67""","""S234H5QKD368C489T""","[""p"", ""p"", … ""p""]","["""", """", … """"]","[""C2"", ""C6"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]","""N:86.AT72.QJT7.AKQ AKQJ97..A42…","[[""86"", ""AT72"", … ""AKQ""], [""AKQJ97"", """", … ""J532""], … [""432"", ""KQ5"", … ""T984""]]"
"""e:\bridge\data\bbo\data\0 …",-1,"""0 A""",2,"""E""","""NS""","""S6TQKH5D5JKAC237Q""","""S23JH67JKD27TQC6A""","""S5789H348QD49C45T""","""S4AH29TAD368C89JK""","[""1C"", ""1S"", … ""p""]","[""Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points"", ""One-level overcall -- 5+ !S; 8-17 HCP; 9-19 total points"", … """"]","[""DA"", ""D7"", … ""S7""]","[1, 1, … 13]","[1, 2, … 4]","""N:KQT6.5.AKJ5.Q732 J32.KJ76.QT…","[[""KQT6"", ""5"", … ""Q732""], [""J32"", ""KJ76"", … ""A6""], … [""A4"", ""AT92"", … ""KJ98""]]"


In [38]:
# takes 20m
deals = [Deal(pbn) for pbn in df['PBN'][:10]]
deals

[Deal('N:86.AT72.QJT7.AKQ AKQJ97..A42.J532 T5.J98643.K95.76 432.KQ5.863.T984'),
 Deal('N:KQT6.5.AKJ5.Q732 J32.KJ76.QT72.A6 9875.Q843.94.T54 A4.AT92.863.KJ98'),
 Deal('N:AK9654.A.QJ6.A54 3.Q876.9432.T976 Q82.JT542.KT8.Q3 JT7.K93.A75.KJ82'),
 Deal('N:A9..AKQT7643.AK3 QJ.AKQJ52.J85.65 K8643.T98.9.J742 T752.7643.2.QT98'),
 Deal('N:J84.AKQ63.K95.K9 K972.T87.AQJ.T62 AT63.42.T64.AJ54 Q5.J95.8732.Q873'),
 Deal('N:AJ743.Q7.A653.Q4 K982.K842.J2.KJ6 65.A.KQT84.97532 QT.JT9653.97.AT8'),
 Deal('N:AQT5.764.AQ2.A86 K982.T.T76.JT752 J6.KJ932.J5.KQ93 743.AQ85.K9843.4'),
 Deal('N:2.AT95.AKQ654.QT A76.KQJ7.T83.985 KQT9.42.J2.AKJ74 J8543.863.97.632'),
 Deal('N:AQJ65.JT.AKT64.9 KT83.K4.J9753.AT 97.A97652.Q8.J62 42.Q83.2.KQ87543'),
 Deal('N:T5.963.AKQ95.A97 97.Q75.J432.T652 KQ86.A842.876.K4 AJ432.KJT.T.QJ83')]

In [61]:
df['PCN'][0:1].to_list()


[[1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4,
  1,
  2,
  3,
  4]]

In [53]:
Contract('1SE=')


Contract("1♠E=")

In [64]:
# Contract(data: 'Union[_dds.contractType, str, None]' = None, *, level: 'Optional[int]' = None, denom: 'Optional[Denom]' = None, declarer: 'Optional[Player]' = None, penalty: 'Optional[Penalty]' = None, result: 'Optional[int]' = None)
# Deal(pbn: 'Optional[str]' = None, first: 'Player' = <Player.north: 0>, trump: 'Denom' = <Denom.nt: 4>, *, complete_deal: 'bool' = False)
help(Board)


NameError: name 'Board' is not defined

In [27]:
# takes 4m
hrs_d = {(pbn,dealer,vul):{'Hands':hands} for pbn,dealer,vul,hands in df[['PBN','Dealer','Vul','Hands']].rows()} # todo: only works if 'Hands' is only key in dict.
len(hrs_d), hrs_d

(900512,
 {('N:86.AT72.QJT7.AKQ AKQJ97..A42.J532 T5.J98643.K95.76 432.KQ5.863.T984',
   'N',
   'None'): {'Hands': [['86', 'AT72', 'QJT7', 'AKQ'],
    ['AKQJ97', '', 'A42', 'J532'],
    ['T5', 'J98643', 'K95', '76'],
    ['432', 'KQ5', '863', 'T984']]},
  ('N:KQT6.5.AKJ5.Q732 J32.KJ76.QT72.A6 9875.Q843.94.T54 A4.AT92.863.KJ98',
   'E',
   'NS'): {'Hands': [['KQT6', '5', 'AKJ5', 'Q732'],
    ['J32', 'KJ76', 'QT72', 'A6'],
    ['9875', 'Q843', '94', 'T54'],
    ['A4', 'AT92', '863', 'KJ98']]},
  ('N:AK9654.A.QJ6.A54 3.Q876.9432.T976 Q82.JT542.KT8.Q3 JT7.K93.A75.KJ82',
   'W',
   'Both'): {'Hands': [['AK9654', 'A', 'QJ6', 'A54'],
    ['3', 'Q876', '9432', 'T976'],
    ['Q82', 'JT542', 'KT8', 'Q3'],
    ['JT7', 'K93', 'A75', 'KJ82']]},
  ('N:A9..AKQT7643.AK3 QJ.AKQJ52.J85.65 K8643.T98.9.J742 T752.7643.2.QT98',
   'N',
   'NS'): {'Hands': [['A9', '', 'AKQT7643', 'AK3'],
    ['QJ', 'AKQJ52', 'J85', '65'],
    ['K8643', 'T98', '9', 'J742'],
    ['T752', '7643', '2', 'QT98']]},
  ('N:J84.AKQ63

In [28]:
# takes 10s for 1m rows and 182MB file size.
bbo_hand_dict_filename = 'bbo_hand_dict.pkl'
bbo_hand_dict_file = dataPath.joinpath(bbo_hand_dict_filename)
with open(bbo_hand_dict_file,'wb') as f:
    pickle.dump(hrs_d,f)
print(f"Saved {bbo_hand_dict_filename}: len:{len(hrs_d)} size:{bbo_hand_dict_file.stat().st_size}")

Saved bbo_hand_dict.pkl: len:900512 size:185537464


In [29]:
def OHE_Hands(hands_bin):
    handsbind = defaultdict(list)
    for h in hands_bin:
        for direction,nesw in zip(mlBridgeLib.NESW,h):
            assert nesw[0] is not None and nesw[1] is not None
            handsbind['_'.join(['HB',direction])].append(nesw[0]) # todo: int(nesw[0],2)) # convert binary string to base 2 int
            #for suit,shdc in zip(mlBridgeLib.SHDC,nesw[1]):
            #    assert shdc is not None
            #    handsbind['_'.join(['HCP',direction,suit])].append(shdc)
    return handsbind

# generic function to augment metrics by suits
def Augment_Metric_By_Suits(metrics,metric,dtype=pl.UInt8):
    for d,direction in enumerate(mlBridgeLib.NESW):
        for s,suit in  enumerate(mlBridgeLib.SHDC):
            metrics = metrics.with_columns(
                metrics[metric].map_elements(lambda x: x[1][d][0],return_dtype=dtype).alias('_'.join([metric,direction])),
                metrics[metric].map_elements(lambda x: x[1][d][1][s],return_dtype=dtype).alias('_'.join([metric,direction,suit]))
            )
    for direction in mlBridgeLib.NS_EW:
        metrics = metrics.with_columns((metrics['_'.join([metric,direction[0]])]+metrics['_'.join([metric,direction[1]])]).cast(dtype).alias('_'.join([metric,direction])))
        for s,suit in  enumerate(mlBridgeLib.SHDC):
            metrics = metrics.with_columns((metrics['_'.join([metric,direction[0],suit])]+metrics['_'.join([metric,direction[1],suit])]).cast(dtype).alias('_'.join([metric,direction,suit])))
    #display(metrics.describe())
    return metrics # why is it necessary to return metrics? Isn't it just df?


In [30]:
import endplay # for __version__
from endplay.parsers import pbn, lin, json
from endplay.types import Deal, Contract, Denom, Player, Penalty, Vul
from endplay.dds import par, calc_all_tables
from endplay.dealer import generate_deals

def display_double_dummy_deals(deals, dd_result_tables, deal_index=0, max_display=4):
    # Display a few hands and double dummy tables
    for dd, rt in zip(deals[deal_index:deal_index+max_display], dd_result_tables[deal_index:deal_index+max_display]):
        deal_index += 1
        print(f"Deal: {deal_index}")
        print(dd)
        rt.pprint()


def calculate_ddtricks_par_scores(df, scores_d, progress=None):

    deals = list(map(Deal,df['Deal'])) # might have to save original deal object or create columns first, trump, complete_deal. recreating from string here.

    # Calculate double dummy and par
    dd_result_tables = calc_double_dummy_deals(deals, progress=progress)

    #display_double_dummy_deals(deals, dd_result_tables, 0, 4)

    # Create dataframe of par scores using double dummy
    pars = [par(rt, Vul.find(v), Player.north) for rt, v in zip(dd_result_tables, df['Vul'])]  # middle arg is board number (if int) otherwise enum vul. Must use Vul.find(v) because uncorrelated to board number.
    par_scores_ns = [parlist.score for parlist in pars]
    par_scores_ew = [-score for score in par_scores_ns]
    par_contracts = [', '.join([str(contract.level) + 'SHDCN'[int(contract.denom)] + contract.declarer.abbr + contract.penalty.abbr + ('' if contract.result == 0 else '+'+str(contract.result) if contract.result > 0 else str(contract.result)) for contract in parlist]) for parlist in pars]
    par_df = pl.DataFrame({'ParScore_NS': par_scores_ns, 'ParScore_EW': par_scores_ew, 'ParContract': par_contracts},orient='row')

    # Create dataframe of double dummy tricks per direction and suit
    DDTricks_df = pl.DataFrame([[s for d in t.to_list() for s in d] for t in dd_result_tables],schema={'_'.join(['DDTricks',d,s]):pl.UInt8 for d in 'NESW' for s in 'SHDCN'},orient='row')

    dd_score_cols = [[scores_d[(level,suit,tricks,vul == 'Both' or (vul != 'None' and direction in vul))] for tricks,vul in zip(DDTricks_df['_'.join(['DDTricks',direction,suit])],df['Vul'])] for direction in 'NESW' for suit in 'SHDCN' for level in range(1, 8)]
    dd_score_df = pl.DataFrame(dd_score_cols, schema=['_'.join(['DDScore', str(l) + s, d]) for d in 'NSEW' for s in 'CDHSN' for l in range(1, 8)])

    return DDTricks_df, par_df, dd_score_df


# todo: could save a couple seconds by creating dict of deals
def calc_double_dummy_deals(deals, batch_size=40, progress=None):
    if isinstance(deals,pl.Series):
        deals = deals.to_list() # this is needed because polars kept ignoring the [b:b+batch_size] slicing. WTF?
    all_result_tables = []
    for b in range(0,len(deals),batch_size):
        if progress:
                percent_complete = int(b*100/len(deals))
                progress.progress(percent_complete,f"{percent_complete}%: {b} of {len(deals)} double dummies calculated.")
        result_tables = calc_all_tables(deals[b:b+batch_size])
        all_result_tables.extend(result_tables)
    if progress:
        progress.progress(100,f"100%: {len(deals)} of {len(deals)} double dummies calculated.")
    return all_result_tables


def constraints(deal):
    return True


def generate_single_dummy_deals(predeal_string, produce, env=dict(), max_attempts=1000000, seed=None, show_progress=True, strict=True, swapping=0):
    
    predeal = Deal(predeal_string)

    deals_t = generate_deals(
        constraints,
        predeal=predeal,
        swapping=swapping,
        show_progress=show_progress,
        produce=produce,
        seed=seed,
        max_attempts=max_attempts,
        env=env,
        strict=strict
        )

    deals = tuple(deals_t) # create a tuple before interop memory goes wonky
    
    return deals, calc_double_dummy_deals(deals)


def calculate_single_dummy_probabilities(deal, produce=100):

    # todo: has this been obsoleted by endplay's calc_all_tables 2nd parameter?
    ns_ew_rows = {}
    for ns_ew in ['NS','EW']:
        s = deal[2:].split()
        if ns_ew == 'NS':
            s[1] = '...'
            s[3] = '...'
        else:
            s[0] = '...'
            s[2] = '...'
        predeal_string = 'N:'+' '.join(s)
        #print(f"predeal:{predeal_string}")

        sd_deals, sd_dd_result_tables = generate_single_dummy_deals(predeal_string, produce, show_progress=False)

        #display_double_dummy_deals(sd_deals, sd_dd_result_tables, 0, 4)
        SDTricks_df = pl.DataFrame([[sddeal.to_pbn()]+[s for d in t.to_list() for s in d] for sddeal,t in zip(sd_deals,sd_dd_result_tables)],schema={'SD_Deal':pl.String}|{'_'.join(['SDTricks',d,s]):pl.UInt8 for d in 'NESW' for s in 'SHDCN'},orient='row')

        for d in 'NSEW':
            for s in 'SHDCN':
                # always create 14 rows (0-13 tricks taken) for combo of direction and suit. fill never-happened with proper index and 0.0 prob value.
                #ns_ew_rows[(ns_ew,d,s)] = dd_df[d+s].to_pandas().value_counts(normalize=True).reindex(range(14), fill_value=0).tolist() # ['Fixed_Direction','Direction_Declarer','Suit']+['SD_Prob_Take_'+str(n) for n in range(14)]
                vc = {ds:p for ds,p in SDTricks_df['_'.join(['SDTricks',d,s])].value_counts(normalize=True).rows()}
                index = {i:0.0 for i in range(14)} # fill values for missing probs
                ns_ew_rows[(ns_ew,d,s)] = list((index|vc).values())

    return SDTricks_df, ns_ew_rows


# def append_single_dummy_results(pbns,sd__d,produce=100):
#     for pbn in pbns:
#         if pbn not in sd_cache_d:
#             sd_cache_d[pbn] = calculate_single_dummy_probabilities(pbn, produce) # all combinations of declarer pair directI. ion, declarer direciton, suit, tricks taken
#     return sd_cache_d


# takes 1000 seconds for 100 sd calcs, or 10 sd calcs per second.
def calculate_sd_probs(df, sd_productions=100, progress=None):

    # calculate single dummy probabilities. if already calculated, use cache.
    sd_cache_d = {}
    sd_dfs_d = {}
    deals = df['Deal']
    for i,deal in enumerate(deals):
        if progress:
            percent_complete = int(i*100/len(deals))
            progress.progress(percent_complete,f"{percent_complete}%: {i} of {len(deals)} single dummies calculated using {sd_productions} samples")
        # st.write(f"{percent_complete}%: {i} of {len(deals)} boards. deal:{deal}")
        if deal not in sd_cache_d:
            sd_dfs_d[deal], sd_cache_d[deal] = calculate_single_dummy_probabilities(deal, sd_productions) # all combinations of declarer pair direction, declarer direciton, suit, tricks taken
    if progress:
        progress.progress(100,f"100%: {len(deals)} of {len(deals)} single dummies calculated.")

    # create single dummy trick taking probability distribution columns
    sd_probs_d = defaultdict(list)
    for deal in deals:
        v = sd_cache_d[deal]
        # st.write(pbn,v)
        for (pair_direction,declarer_direction,suit),probs in v.items():
            for i,t in enumerate(probs):
                sd_probs_d['_'.join(['Probs',pair_direction,declarer_direction,suit,str(i)])].append(t)
    # st.write(sd_probs_d)
    sd_probs_df = pl.DataFrame(sd_probs_d,orient='row')
    return sd_dfs_d, sd_cache_d, sd_probs_df


# calculate dict of contract result scores. each column contains (non-vul,vul) scores for each trick taken. sets are always penalty doubled.
def calculate_scores():

    scores_d = {}
    suit_to_denom = [Denom.clubs, Denom.diamonds, Denom.hearts, Denom.spades, Denom.nt]
    for suit_char in 'SHDCN':
        suit_index = 'CDHSN'.index(suit_char) # [3,2,1,0,4]
        denom = suit_to_denom[suit_index]
        for level in range(1,8): # contract level
            for tricks in range(14):
                result = tricks-6-level
                # sets are always penalty doubled
                scores_d[(level,suit_char,tricks,False)] = Contract(level=level,denom=denom,declarer=Player.north,penalty=Penalty.passed if result>=0 else Penalty.doubled,result=result).score(Vul.none)
                scores_d[(level,suit_char,tricks,True)] = Contract(level=level,denom=denom,declarer=Player.north,penalty=Penalty.passed if result>=0 else Penalty.doubled,result=result).score(Vul.both)

    # create score dataframe from dict
    sd = defaultdict(list)
    for suit in 'SHDCN':
        for level in range(1,8):
            for i in range(14):
                sd['_'.join(['Score',str(level)+suit])].append([scores_d[(level,suit,i,False)],scores_d[(level,suit,i,True)]])
    # st.write(all_scores_d)
    scores_df = pl.DataFrame(sd,orient='row')
    # scores_df.index.name = 'Taken'
    return scores_d, scores_df


def calculate_sd_expected_values(df,sd_cache_d,scores_df):
    # create dict of expected values (probability * score)
    exp_d = defaultdict(list)
    deal_vul = zip(df['Deal'],df['Vul'])
    for deal,vul in deal_vul:
        #st.write(deal,vul)
        for (pair_direction,declarer_direction,suit),probs in sd_cache_d[deal].items():
            is_declarer_vul = vul == 'Both' or (vul != 'None' and declarer_direction in vul)
            #st.write(pair_direction,declarer_direction,suit,probs,is_declarer_vul)
            for level in range(1,8):
                #st.write(scores_d['_'.join(['Score',str(level)+suit])][is_declarer_vul])
                exp_d['_'.join(['Exp',pair_direction,declarer_direction,suit,str(level)])].append(sum([prob*score[is_declarer_vul] for prob,score in zip(probs,scores_df['_'.join(['Score',str(level)+suit])])]))
            #st.write(exp_d)
    #st.write(exp_d)
    sd_exp_df = pl.DataFrame(exp_d,orient='row')
    return sd_exp_df


# create columns containing the 1) the name of the column having the max expected value. 2) max expected value 3) contract having the max expected value.
def create_best_contracts(df):
    besties = []
    for d in df.to_dicts():
        exp_tuples = tuple([(v,k) for k,v in d.items()])
        ex_tuples_sorted = sorted(exp_tuples,reverse=True)
        best_contract_tuple = ex_tuples_sorted[0]
        best_contract_split = best_contract_tuple[1].split('_') # split column name into parts
        best_contract = best_contract_split[4]+best_contract_split[3]+best_contract_split[2]
        besties.append([best_contract_tuple[1],best_contract_tuple[0],best_contract_tuple[0] if best_contract_tuple[1][-5] in ['N','S'] else -best_contract_tuple[0],best_contract])
    return besties


def calculate_best_contracts(sd_exp_df):
    sd_best_contract_l = create_best_contracts(sd_exp_df) #sd_exp_df.to_pandas().apply(create_best_contracts,axis='columns')
    sd_best_contract_df = pl.DataFrame(sd_best_contract_l,schema=['ExpMaxScore_Col','Exp_Max','ExpMaxScore_NS','BestContract'],orient='row')
    return sd_best_contract_df


def convert_contract_to_contract(df):
    return df['Contract'].str.to_uppercase().str.replace('♠','S').str.replace('♥','H').str.replace('♦','D').str.replace('♣','C').str.replace('NT','N')


# None is used instead of pl.Null because pl.Null becomes 'Null' string in pl.String columns. Not sure what's going on but the solution is to use None.
def convert_contract_to_declarer(df):
    return [None if c == 'PASS' else c[2] for c in df['Contract']] # extract declarer from contract


def convert_declarer_to_DeclarerName(df):
    return [None if d is None else df[d][i] for i,d in enumerate(df['Declarer'])] # extract declarer name using declarer direction as the lookup key


def convert_contract_to_result(df):
    return [None if c == 'PASS' else 0 if c[-1] in ['=','0'] else int(c[-1]) if c[-2] == '+' else -int(c[-1]) for c in df['Contract']] # create result from contract


def convert_contract_to_tricks(df):
    return [None if c == 'PASS' else int(c[0])+6+r for c,r in zip(df['Contract'],df['Result'])] # create tricks from contract and result


def convert_contract_to_DDTricks(df):
    return [None if c == 'PASS' else df['_'.join(['DDTricks',d,c[1]])][i] for i,(c,d) in enumerate(zip(df['Contract'],df['Declarer']))] # extract double dummy tricks using contract and declarer as the lookup keys


def convert_score_to_score(df):
    scores = []
    for d in df.to_dicts():
        score_split = d['Score'].split()
        assert len(score_split) == 2, f"score_split:{score_split}"
        assert score_split[0] in ['NS','EW'], f"score_split:{score_split[0]}"
        assert score_split[1][0] == '-' or str.isdigit(score_split[1][0]), f"score_split:{score_split[1]}"
        score_split_direction = score_split[0]
        score_split_value = score_split[1]
        score_value = -int(score_split_value) if score_split_value[0] == '-' else int(score_split_value)
        scores.append(score_value if score_split_direction == 'NS' else -score_value)
    return scores


def create_augmented_df(df):
    #df = df.clone()
    df = df.rename({'North':'N','East':'E','South':'S','West':'W'}) # todo: is this really better?

    df = df.with_columns(
        pl.Series('Contract',convert_contract_to_contract(df),pl.String,strict=False), # can have nulls or Strings
    )
    df = df.with_columns(
        pl.Series('Declarer',convert_contract_to_declarer(df),pl.String,strict=False), # can have nulls or Strings
    )
    df = df.with_columns(
        pl.Series('DeclarerName',convert_declarer_to_DeclarerName(df),pl.String,strict=False), # can have nulls or Strings
        pl.Series('Result',convert_contract_to_result(df),pl.Int8,strict=False), # can have nulls or Int8
    )
    df = df.with_columns(
        pl.Series('Tricks',convert_contract_to_tricks(df),pl.UInt8,strict=False), # can have nulls or UInt8
        pl.Series('DDTricks',convert_contract_to_DDTricks(df),pl.UInt8,strict=False), # can have nulls or UInt8
        pl.Series('Score_NS',convert_score_to_score(df),pl.Int16),
    )
    df = df.with_columns(
        pl.Series('ParScore_Diff_NS',(df['Score_NS']-df['ParScore_NS']),pl.Int16),
        # needs to have .cast(pl.Int8) because left and right are both UInt8 which goofs up the subtraction.
        pl.Series('DDTricks_Diff',(df['Tricks'].cast(pl.Int8)-df['DDTricks'].cast(pl.Int8)),pl.Int8,strict=False), # can have nulls or Int8
        pl.Series('ExpMaxScore_Diff_NS',(df['Score_NS']-df['ExpMaxScore_NS']),pl.Float32),
    )
    df = df.with_columns(
        pl.Series('ParScore_Diff_EW',-df['ParScore_Diff_NS'],pl.Int16), # used for open-closed room comparisons
        pl.Series('ExpMaxScore_Diff_EW',-df['ExpMaxScore_Diff_NS'],pl.Float32), # used for open-closed room comparisons
    )
    return df


In [31]:

def AugmentHands(df,hrs_d):

    # create column of Hands expressed in binary.
    if 'Hands_Bin' in df.columns:
        print('Hands_Bin already exists. skipping...')
    else:
        # takes 18m
        t = time.time()
        hbs_l = [mlBridgeLib.HandsToBin(hands) for hands in df['Hands']]
        df = df.with_columns(pl.Series('Hands_Bin',hbs_l,pl.Object))
        del hbs_l
        print(f"Time to create Hands_Bin: {time.time()-t} seconds")
    print(df[['Hands','Hands_Bin']])

    # Create one hot encoding, length 52, for each direction's hand.
    # todo: one hot encode each direction's hand? Leaving as binary string for now.
    if 'HB_N' in df.columns:
        print('HB_N already exists. skipping...')
    else:
        # takes 30s
        t = time.time()
        hands_bin_d = OHE_Hands(df['Hands_Bin'])
        hands_bin_df = pl.DataFrame(hands_bin_d)
        df = pl.concat([df,hands_bin_df],how='horizontal')
        del hands_bin_df,hands_bin_d
        print(f"Time to create HB_N: {time.time()-t} seconds")
    print(df[['Hands','HB_N','HB_E','HB_S','HB_W']])

    # convert HB to OHE (One Hot Encoding) in the form of C_[NESW][SHDC][Rank]
    if 'C_NSA' in df.columns:
        print('C_NSA already exists. skipping...')
    else:
        # takes 10m
        t = time.time()
        c_ds = [[c for h in t for c in h[0]] for t in df['Hands_Bin']]
        schema = {f"C_{d}{s}{c}":pl.Boolean for d in mlBridgeLib.NESW for s in mlBridgeLib.SHDC for c in mlBridgeLib.ranked_suit}
        ohe_hands_df = pl.DataFrame(c_ds,orient='row',schema=schema)
        df = pl.concat([df,ohe_hands_df],how='horizontal')
        del ohe_hands_df,c_ds
        print(f"Time to create C_NSA: {time.time()-t} seconds")
    print(df.select(pl.col(['Hands',r'^HB_[NESW]$',r'^C_[NESW][SHDC][2-9TJQKA]$'])))

    # Compute HCPs from Hands. Validate against any existing HCP column.
    if 'HCP_N_C' in df.columns:
        print('HCP_N_C already exists. skipping...')
    else:
        # takes 10m
        t = time.time()
        df = df.with_columns(pl.Series('HCP',[mlBridgeLib.HandsToHCP(h) for h in df['Hands']],dtype=pl.Object))
        df = Augment_Metric_By_Suits(df,'HCP')
        print(f"Time to create HCP_N_C: {time.time()-t} seconds")
    print(df.select(pl.col(r'^HCP_[NESW]$','^HCP_(NS|EW)$','^HCP_[NESW]_[SHDC]$')))

    # Compute quick tricks from Hands
    if 'QT_N_C' in df.columns:
        print('QT_N_C already exists. skipping...')
    else:
        # takes 10m
        t = time.time()
        df = df.with_columns(pl.Series('QT',[mlBridgeLib.HandsToQT(h) for h in df['Hands']],dtype=pl.Object))
        df = Augment_Metric_By_Suits(df,'QT',pl.Float32)
        print(f"Time to create QT_N_C: {time.time()-t} seconds")
    print(df.select(pl.col(r'^QT_[NESW]$',r'^QT_(NS|EW)$',r'^QT_[NESW]_[SHDC]$')))

    # Compute suit lengths from Hands
    if 'SL_N_C' in df.columns:
        print('SL_N_C already exists. skipping...')
    else:
        # takes 9m
        t = time.time()
        df = df.with_columns(pl.Series('SL',[mlBridgeLib.HandsToSuitLengths(h) for h in df['Hands']],dtype=pl.Object))
        df = Augment_Metric_By_Suits(df,'SL')
        print(f"Time to create SL_N_C: {time.time()-t} seconds")
    print(df.select(pl.col(r'^SL_[NESW]_[SHDC]$',r'^SL_(NS|EW)_[SHDC]$')))

    if 'SL_N_CDHS' in df.columns:
        print('SL_N_CDHS already exists. skipping...')
        assert 'SL_N_CDHS_SJ' in df.columns and 'SL_N_ML' in df.columns and 'SL_N_ML_SJ' in df.columns and 'SL_N_ML_I' in df.columns and 'SL_N_ML_I_SJ' in df.columns
    else:
        # takes 17m15s-22m
        t = time.time()
        for d in 'NESW':
            cdhs_l = df[[f"SL_{d}_{s}" for s in 'CDHS']].rows() # CDHS suit lengths
            ml_li_l = [sorted([(l,i) for i,l in enumerate(r)],reverse=True) for r in df[[f"SL_{d}_{s}" for s in 'CDHS']].rows()] # (length,index) ex: (4,3),(4,0),(3,1),(2,2)
            ml_l = [[t2[0]  for t2 in t4] for t4 in ml_li_l] # most-to-least lengths
            ml_i_l = [[t2[1]  for t2 in t4] for t4 in ml_li_l] # column indices of most-to-least lengths
            df = df.with_columns(
                pl.Series(f'SL_{d}_CDHS',cdhs_l,pl.Array(pl.UInt8,shape=(4,))), # CDHS suit lengths
                pl.Series(f'SL_{d}_CDHS_SJ',['-'.join(map(str,r)) for r in cdhs_l],pl.String), # CDHS suit lengths stringized and joined
                pl.Series(f"SL_{d}_ML",ml_l,pl.Array(pl.UInt8,shape=(4,))), # most-to-least suit lengths
                pl.Series(f"SL_{d}_ML_SJ",['-'.join(map(str,r)) for r in ml_l],pl.String), # most-to-least suit lengths stringized and joined
                pl.Series(f"SL_{d}_ML_I",ml_i_l,pl.Array(pl.UInt8,shape=(4,))), # column indices of most-to-least
                pl.Series(f"SL_{d}_ML_I_SJ",['-'.join(map(str,r)) for r in ml_i_l],pl.String), # column indices of most-to-least stringized and joined
            )
        print(f"Time to create SL_N_CDHS: {time.time()-t} seconds")
    print(df.select(pl.col(r'^SL_[NESW]_CDHS$',r'^SL_[NESW]_CDHS_SJ$',r'^SL_[NESW]_ML$',r'^SL_[NESW]_ML_SJ$',r'^SL_[NESW]_ML_I$',r'^SL_[NESW]_ML_I_SJ$')))

    # Calculate distribution points using 3-2-1 system.
    if 'DP_N_C' in df.columns:
        print('DP_N_C already exists. skipping...')
    else:
        # takes 4m30s
        t = time.time()
        df = df.with_columns(pl.Series('DP',[mlBridgeLib.HandsToDistributionPoints(h) for h in df['Hands']],dtype=pl.Object))
        df = Augment_Metric_By_Suits(df,'DP')
        print(f"Time to create DP_N_C: {time.time()-t} seconds")
    print(df.select(pl.col('^DP_[NESW]$',r'^DP_(NS|EW)$',r'^DP_[NESW]_[SHDC]$')))

    if 'SL_Max_NS' in df.columns:
        print('SL_Max_NS already exists. skipping...')
    else:
        # takes 15s
        t = time.time()
        sl_cols = [('_'.join(['SL_Max',d]),['_'.join(['SL',d,s]) for s in mlBridgeLib.SHDC]) for d in mlBridgeLib.NS_EW]
        # Create columns containing column names of the NS,EW longest suit.
        for d in sl_cols:
            df = df.with_columns(pl.Series(d[0],[d[1][l.index(max(l))] for l in df[d[1]].rows()])) #.cast(pl.Categorical)) #.alias(d[0])) # defaults to object so need string or category
        print(f"Time to create SL_Max_NS: {time.time()-t} seconds")
    print(df[[c[0] for c in sl_cols]+[c for l in sl_cols for c in l[1]]])

    if 'Par' in df.columns:
        print('Par already exists. skipping...')
    else:
        # takes 15s
        t = time.time()
        Pars_l = [hrs_d[pdv]['Par'] for pdv in df[('PBN','Dealer','Vul')].rows()]
        df = df.with_columns(pl.Series('Par',Pars_l,pl.Object)) # todo: specify correct dtype instead of object
        print(f"Time to create Par: {time.time()-t} seconds")
    print(df.describe())

    # Create columns of Par_Score. Give it pl.Categorical dtype.
    if 'ParScore' in df.columns:
        print('Par_Score already exists. skipping...')
    else:
        # takes 10s
        t = time.time()
        df = df.with_columns(pl.Series('ParScore',[p[0] for p in Pars_l],dtype=pl.Int16)) # is categorical the right dtype?
        print(f"Time to create Par_Score: {time.time()-t} seconds")
    print(df.describe())

    if 'LoTT' in df.columns:
        print('LoTT already exists. skipping...')
    else:
        # takes 1m30s
        t = time.time()
        LoTT_l = [mlBridgeLib.LoTT_SHDC(r[0],r[1]) for r in df[['DDmakes','SL']].rows()]
        df = df.with_columns(
            pl.Series('LoTT',LoTT_l,dtype=pl.Object),
            pl.Series('LoTT_Tricks',[t for t,l,v in LoTT_l],dtype=pl.UInt8),
            pl.Series('LoTT_Suit_Length',[l for t,l,v in LoTT_l],dtype=pl.UInt8),
            pl.Series('LoTT_Variance',[v for t,l,v in LoTT_l],dtype=pl.Int8)
        )
        del LoTT_l
        print(f"Time to create LoTT: {time.time()-t} seconds")
    print(df[['Hands','DDmakes','SL','LoTT_Tricks','LoTT_Suit_Length','LoTT_Variance','Par','ParScore']])

    # Create column of contract types by partnership by suit. e.g. CT_NS_C.
    if 'CT_NS_C' in df.columns:
        print('CT_NS_C already exists. skipping...')
    else:
        # takes 1m30s
        t = time.time()
        contract_types_d = mlBridgeLib.CategorifyContractTypeBySuit(df['DDmakes']) # pl.Categorical?
        contract_types_s = pl.DataFrame(contract_types_d)
        df = pl.concat([df,contract_types_s],how='horizontal')
        del contract_types_s,contract_types_d
        print(f"Time to create CT_NS_C: {time.time()-t} seconds")
    print(df.describe())

    # Create columns of contract types by partnership by suit by contract. e.g. CT_NS_C_Game
    if 'CT_NS_C_Game' in df.columns:
        print('CT_NS_C_Game already exists. skipping...')
    else:
        # takes 5s
        t = time.time()
        contract_types_d = mlBridgeLib.CategorifyContractTypeByDirection(df) # pl.Categorical?
        contract_types_s = pl.DataFrame(contract_types_d)
        df = pl.concat([df,contract_types_s],how='horizontal')
        del contract_types_s,contract_types_d
        print(f"Time to create CT_NS_C_Game: {time.time()-t} seconds")
    print(df.describe())

    # Create columns of vulnerability by partnership.
    if 'Vul_NS' in df.columns:
        print('Vul_NS already exists. skipping...')
    else:
        # takes 5s
        t = time.time()
        df = df.with_columns(
            pl.Series('Vul_NS',df['Vul'].is_in(['N_S','Both']),dtype=pl.Boolean),
            pl.Series('Vul_EW',df['Vul'].is_in(['E_W','Both']),dtype=pl.Boolean)
        )
        print(f"Time to create Vul_NS: {time.time()-t} seconds")
    print(df.describe())

    # takes 1m
    # todo: move to acbl_club_results_hand_records_clean.ipynb
    df = df.filter(pl.col('Board') <= 255)

    # change dtypes where needed
    df = df.with_columns(
        pl.col('Board').cast(pl.UInt8),
        #pl.col('Dealer').cast(pl.String),
        pl.col('Vul').cast(pl.String),
        #pl.col('board_record_string').cast(pl.String),
    )

    return df

def AugmentACBLHandRecords(df,hrs_d):
    df = AugmentHands(df,hrs_d)

    # Columns unique to ACBL Club results hand records
    # takes 5s
    if 'game_date' in df.columns:
        df = df.with_columns(pl.Series('Date',df['game_date'].str.strptime(pl.Date,'%Y-%m-%d %H:%M:%S')))
    # takes 5s
    if 'hand_record_id' in df.columns:
        df = df.with_columns(
            pl.col('hand_record_id').cast(pl.String),
        )
    return df


In [32]:
df = AugmentHands(df,hrs_d)
df


Time to create Hands_Bin: 1196.7731051445007 seconds
shape: (14_224_835, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ Hands                           ┆ Hands_Bin                       │
│ ---                             ┆ ---                             │
│ list[list[str]]                 ┆ object                          │
╞═════════════════════════════════╪═════════════════════════════════╡
│ [["86", "AT72", … "AKQ"], ["AK… ┆ (('000000101000010001001000010… │
│ [["KQT6", "5", … "Q732"], ["J3… ┆ (('011010001000000000000010001… │
│ [["AK9654", "A", … "A54"], ["3… ┆ (('110001001110010000000000000… │
│ [["A9", "", … "AK3"], ["QJ", "… ┆ (('100001000000000000000000001… │
│ [["J84", "AKQ63", … "K9"], ["K… ┆ (('000100100010011100000100100… │
│ …                               ┆ …                               │
│ [["J6", "KQJ8", … "AQJ64"], ["… ┆ (('000100001000001110010000000… │
│ [["AT5", "KQJ", … "AQ863"], ["… ┆ (('100010000100001110000000001… │
│ [["J2", "KQ5

KeyboardInterrupt: 

In [ ]:
df.select(pl.col(['Hands',r'^HB_[NESW]$',r'^C_[NESW][SHDC][2-9TJQKA]$']))


In [ ]:
print('Program elapsed time in seconds:', time.time()-program_start_time)